In [8]:
#libraries
from bs4 import BeautifulSoup, element
import urllib
import requests
import pandas as pd
import numpy as np
import lyricsgenius


In [9]:
genius = lyricsgenius.Genius("rPVs-pFT7GhBfTxhpu-ISnNCcvCsbRt8wIwhkkEXovrADgSBQfkndQW7Ge22R5Ts")
song = genius.search_song("Blinding Lights", "The Weeknd")
print(song.lyrics)

Searching for "Blinding Lights" by The Weeknd...
Done.
249 ContributorsTranslationsTürkçeSvenskaEspañolSlovenčinaРусский (Russian)PortuguêsPolskiNorsk日本語 (Japanese)Italianoעברית (Hebrew)DeutschFrançaisفارسیNederlandsDanskShqipالعربيةsrpskiBlinding Lights Lyrics“Blinding Lights” is the second single to The Weeknd’s fourth studio album After Hours, following the record’s lead single “Heartless,” which released two days prior on November 27, 2019. The track finds Abel… Read More [Intro]
Yeah

[Verse 1]
I've been tryna call
I've been on my own for long enough
Maybe you can show me how to love, maybe
I'm goin' through withdrawals
You don't even have to do too much
You can turn me on with just a touch, baby

[Pre-Chorus]
I look around and
Sin City's cold and empty (Oh)
No one's around to judge me (Oh)
I can't see clearly when you're gone

[Chorus]
I said, ooh, I'm blinded by the lights
No, I can't sleep until I feel your touch
I said, ooh, I'm drowning in the night
Oh, when I'm like this, yo

C:\Users\Monado\AppData\Roaming\Python\Python312\site-packages\lyricsgenius\genius.py:468: FutureWarning: The constructor signature will change in a future version. It will change to Song(lyrics, body) instead of Song(client, json_dict, lyrics).
  song = Song(self, song_info, lyrics)


In [10]:
print(type(song.lyrics))

<class 'str'>


In [12]:
print(song.lyrics[0:100])

249 ContributorsTranslationsTürkçeSvenskaEspañolSlovenčinaРусский (Russian)PortuguêsPolskiNorsk日本語 (
